In [11]:
import sys
import os
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
from configs import configs
import importlib
from generator import GenThreadSafe, SemisupWrapper
from batch_fetcher import BatchFetcher

In [17]:
def make_batch_fetcher(typ, num_examples,data_name, data_file):
    data_module = importlib.import_module("notebooks_src.load_data.datasets." + data_name+".get_data")
    ims, labels = data_module.get_data(type_=typ, data_file=data_file)
    return BatchFetcher(ims,labels)

In [18]:
def get_generator(typ, data_path=None, data_name="climate", batch_size=None,num_ims=-1, mode="supervised" ):
    
    if data_path is None:
        data_path = configs["data_file"]
    num_ims = configs["num_"+ typ+"_ims"]
    batch_fetcher = make_batch_fetcher(num_examples=num_ims,typ=typ,
                                       data_name=data_name, data_file=data_path)
    
    if batch_size is None:
        batch_size = configs["batch_size"]
    generator = GenThreadSafe(batch_fetcher, 
                              batch_size = batch_size,
                              typ=typ, 
                              tf_mode=True, 
                              num_ex = num_ims) 
    
    if mode == "semi_supervised":
        generator = SemisupWrapper(generator)
        
    return generator